In [1]:
%load_ext autoreload
%autoreload 2

import util
import config


project_id = 119867
pool_id = 36425808

In [2]:
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor

pool = ThreadPoolExecutor(12)

In [3]:
import pandas as pd
from collections import Counter

In [4]:
from tqdm.auto import tqdm

In [9]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200", basic_auth=('elastic', 'changeme'))
es_client.info()

ObjectApiResponse({'name': 'a56e8670f4e9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'axIx5HhRQY6iIouQ_OkZ1A', 'version': {'number': '8.5.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c1310c45fc534583afe2c1c03046491efba2bba2', 'build_date': '2022-11-09T21:02:20.169855900Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
def save_hashes(attachment_id, doc):
    es_client.index(index="attachment", id=attachment_id, document=doc)

def find_duplicates(doc):
    keys = ['dhash', 'phash', 'md5']
    should = [{"term": {k: doc[k]}} for k in keys]
    query = {'bool': {'should': should}}

    response = es_client.search(query=query, index='attachment')
    return response['hits']

def has_duplicates(doc):
    response = find_duplicates(doc)
    return len(response['hits']) > 0

In [11]:
def get_available_suits(pool_id, last_id=None, status='SUBMITTED'):
    tasks_url = f'api/v1/assignments'

    params = {
        'status': status,
        'pool_id': pool_id,
        'sort': 'id',
        'limit': 100,
    }

    if last_id is not None:
        params['id_gt'] = last_id
    
    suits = []
    ids = set()

    while True:
        resp = util.toloka_get(tasks_url, params).json()

        items = resp['items']
        suits.extend(items)
        ids.update({i['id'] for i in items})

        print(f'added {len(items)}, now have {len(suits)} suits ({len(ids)} unique)')

        if resp['has_more'] == False:
            break

        params['id_gt'] = items[-1]['id']

    return suits, items[-1]['id']

In [11]:
all_suits, last_id = get_available_suits(pool_id)#, last_id='00022bd050--6378039edb0e8f36d8eea3e5')

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 100, now have 900 suits (900 unique)
added 100, now have 1000 suits (1000 unique)
added 100, now have 1100 suits (1100 unique)
added 100, now have 1200 suits (1200 unique)
added 100, now have 1300 suits (1300 unique)
added 100, now have 1400 suits (1400 unique)
added 100, now have 1500 suits (1500 unique)
added 100, now have 1600 suits (1600 unique)
added 100, now have 1700 suits (1700 unique)
added 100, now have 1800 suits (1800 unique)
added 100, now have 1900 suits (1900 unique)
added 100, now have 2000 suits (2000 unique)


In [12]:
last_id

'00022bd050--638313db8c39c02e11a44110'

In [13]:
image_urls = []

for suite in all_suits:
    for sol in suite['solutions']:
        image_urls.append(sol['output_values']['image'])
        
len(image_urls)

2000

In [17]:
def download_resize_upload(attachment_id):
    _, hashes = util.download_image_and_calc_hashes(attachment=attachment_id)

    if hashes is None:
        return (attachment_id, None)

    image_url = util.make_upload_thumbnail(pool_id, attachment_id)

    return (attachment_id, (hashes, image_url))

In [21]:
from PIL import Image

In [22]:
img = Image.open('images/eu.5b35e616-208d-495f-99ad-6454955feb32')

'RGB'

In [18]:
download_resize_upload(image_urls[0])

images/eu.5b35e616-208d-495f-99ad-6454955feb32 already exist


('eu.5b35e616-208d-495f-99ad-6454955feb32',
 ({'dhash': '111ee8bc3ef83818',
   'phash': 'd4e86ab5b5d2c02e',
   'whash': 'c080fcfefefe8800',
   'md5': 'b4a74f7ad3d474df4e9ad4418db7cc21'},
  'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.5b35e616-208d-495f-99ad-6454955feb32.jpg'))

In [26]:
image_hashes = util.map_progress(pool, image_urls, download_resize_upload)

  0%|          | 0/2000 [00:00<?, ?it/s]

images/eu.6a1a87ba-5690-40ac-83fe-62f9992ef5ce already exist
images/eu.5b35e616-208d-495f-99ad-6454955feb32 already exist
images/eu.c703be96-7957-4844-8eac-ad6b726a3ceb already exist
images/eu.a0f1bf35-7192-40e8-a274-4ba8f4968caa already exist
images/eu.f91f6108-a822-4ada-9f1c-759b606ad02c already exist
images/eu.4de10a96-9754-4a03-8a07-91794d6986a4 already exist
images/eu.4fa71998-aeda-4911-a5fa-359bf2f81268 already exist
images/eu.b82b03a9-c97f-4232-b65b-0511e8512fd4 already exist
images/eu.7eb5c8be-05ac-444f-b656-f5f44846f46c already exist
images/eu.536e2b8c-1f73-40b8-9a71-fe19b8aa3ad6 already exist
images/eu.f33d998d-532f-4c5f-ba36-16ef55b2fb6c already exist
images/eu.aae4cba5-e615-45d7-b983-39ffeb7b6b42 already exist
images/eu.02723b35-1f9f-423a-9c0f-a4c0b97c11dd already exist
images/eu.aa208ffa-d969-4038-bd42-7415a3f2746c already exist
images/eu.2af14c69-c917-4fec-b59c-4c3eb6f92543 already exist
images/eu.ec71cfb3-03db-4d81-802b-219308a4c304 already exist
images/eu.fe7d440d-c72f-

images/eu.36551d62-c1c7-4220-8eea-557f1dd32a29 already exist
images/eu.93477a19-edd1-43a9-bff5-7d8b61d5ea25 already exist
images/eu.34afacf0-a9c4-4564-9bd0-ce5f0e73d9d1 already exist
images/eu.cf547526-f09d-48eb-97c1-9022d7fb32f5 already exist
images/eu.79e3f20b-45d8-4a66-a089-05fcfd32d681 already exist
images/eu.6936233e-d6b5-49df-8355-de9bd06a9c50 already exist
images/eu.86cd33e6-fa2f-4520-8a78-7204201381ea already exist
images/eu.c2c84b61-6c1c-47d8-9ae1-088dd5be3301 already exist
images/eu.933a000f-249b-415c-bbd3-4d318ea9bc67 already exist
images/eu.12f57a2b-5046-44f3-b66f-70a8eecee2be already exist
images/eu.06ce497a-ad95-4bd1-a949-e6e3593d887b already exist
images/eu.ec17a7be-5260-4177-bb9e-5a1b289c7289 already exist
images/eu.2152a226-4600-4f2b-8ae5-e0ac16de4d6a already exist
images/eu.390da0d7-2e70-431e-9ea4-a75fcc47e80d already exist
images/eu.e74b1630-8b49-4096-b031-2a2e72f94025 already exist
images/eu.a49d3764-6279-4f01-940e-bcde0c3d799b already exist
images/eu.752cde55-498b-

images/eu.5487f4dd-fbf9-455c-8cd0-0788d1854d94 already exist
images/eu.60552210-0dd2-4ee5-9f8a-b6846a6662dd already exist
images/eu.c4519eaf-39cf-4eb4-a3b4-b89a7c45397f already exist
images/eu.f26340ab-fbbe-4edf-a471-f3dd41b0c120 already exist
images/eu.055923b6-cfb3-4739-b891-d67d54ecbd53 already exist
images/eu.96311794-86e8-4091-b0d9-a3b03fd28c9e already exist
images/eu.c2c9b610-fc1f-4637-8531-d7d474d498dd already exist
images/eu.064fbdb4-940e-48cf-94e9-b88d0d4cc263 already exist
images/eu.5d625dd7-1c36-4288-9313-bd6f24614dcb already exist
images/eu.235895a5-9537-4cc8-aac6-3f1138efaa95 already exist
images/eu.d2f3dfd9-8536-4535-8890-8e11b7aad7de already exist
images/eu.9a5029f1-a1d9-4fe5-a25b-03346a72eb3c already exist
images/eu.65573d05-613f-4450-8c0f-f47e129cf170 already exist
images/eu.44359ad9-8f1f-47a5-97af-ebb47a527813 already exist
images/eu.e7b04bbd-2c2a-4a37-ac21-0c187274f2f7 already exist
images/eu.8388c85a-8250-4038-bd4a-dbf4201b59e8 already exist
images/eu.116835e3-2396-

images/eu.c10978b7-3431-4b0b-9784-7f8bd9df25a0 already exist
images/eu.b2b04f8c-2a3a-4e47-ba32-0f7ca02d9a4e already exist
images/eu.9613f28b-f212-4982-8e32-8c8771f8de9e already exist
images/eu.d9035dec-321d-4b36-97ed-3c599e4849a1 already exist
images/eu.573644ec-9b2f-4e8a-b4c9-df33e88eca74 already exist
images/eu.7eb843da-8738-47f4-97e1-5668562caebc already exist
images/eu.efb85edd-cf60-4d92-9710-312bcd01e28b already exist
images/eu.745d6d5c-ada0-4ec8-aab2-af941a3d86f0 already exist
images/eu.888e5e0d-5c43-48a0-9026-5ebea707dfc9 already exist
images/eu.1b13d81b-de1d-43f1-8973-6351b2eac4db already exist
images/eu.8119c33c-e675-4893-bdc4-bf2f3eeba88c already exist
images/eu.d47c4c78-ffa6-4795-a2da-6a0856787fad already exist
images/eu.247cf258-9355-462a-b385-cee490a677d7 already exist
images/eu.69b1daea-f156-4e69-bea8-8921e6b5d053 already exist
images/eu.45737e40-1edb-46e7-a0ab-238cc275b969 already exist
images/eu.bdf564df-e035-4be7-b221-8c1bfc847724 already exist
images/eu.c658feff-b0c1-

...

images/eu.16c8af18-7780-4819-863b-cc28d1b1973c already exist
images/eu.256bbc0b-dc52-4320-965b-e4daa94df0eb already exist
images/eu.d8cd836e-0791-4e6d-89d6-e8f9e5cd95a2 already exist
images/eu.385846f3-dc20-4a40-acf7-02797cccaf2f already exist
images/eu.bedc9692-4910-47b1-aa78-8df243dc6ff9 already exist
images/eu.31bb1946-e912-41f0-85a3-b57e330cb489 already exist
images/eu.b85e9eef-a72b-4970-9f29-98bc2517d11d already exist
images/eu.a8d3f9af-b8b1-4606-bdae-e18aecf3eaae already exist
images/eu.d1669c1b-63a8-4023-8a22-307056d496d9 already exist
images/eu.e781ab23-1ed6-47a5-a544-b53045b3c182 already exist
images/eu.156455fe-aa81-4d38-9220-c91aff9dbbcd already exist
images/eu.f326ee32-8bdf-4860-a8b6-c9582c443b38 already exist
images/eu.e7fe8ef5-d981-4d80-9ff6-62eeb505a5b1 already exist
images/eu.4059cb4a-5d95-4955-9150-35da3e375f13 already exist
images/eu.ed774b95-5c7b-4c78-8c9b-3b03992f4530 already exist
images/eu.6a9987d4-3e09-4af4-a654-e18d13067b30 already exist
images/eu.64bdff00-f40f-

images/eu.8f4ff145-7d4a-477a-9d16-b04dcc6393fb already exist
images/eu.94bf76b4-6f05-425a-961b-8c00ec96ae3b already exist
images/eu.09a8c820-0b8b-44e0-b739-fa2d51855fcb already exist
images/eu.b8c3d6ab-7297-43a1-8c3a-e18c80918691 already exist
images/eu.c9aa4769-169d-4f79-b2d4-8bf7cd86b045 already exist
images/eu.5aae738b-3f4d-42fc-a7ea-e49354c368a3 already exist
images/eu.74522bde-369e-4440-814c-cfaf1221644b already exist
images/eu.08127579-1880-4476-a74f-add97344488c already exist
images/eu.82dd4b47-c004-460d-b620-d8201907a720 already exist
images/eu.9f6a151c-f579-4cc6-8c7a-f60f228ecfaf already exist
images/eu.b197b7c6-4edc-49e9-8453-0fe9366e69b1 already exist
images/eu.9056036e-59e3-4bb2-b8ea-d0a3d86c9750 already exist
images/eu.779e25d4-1c6f-4e0a-8f6b-9f5778bf219c already exist
images/eu.c7681f7d-2dc9-4b13-a54a-71de82333992 already exist
images/eu.9a8b61b8-dce1-479b-b218-f90e5d2da5b2 already exist
images/eu.34ddf52d-d33b-48ce-906c-cfd44b007729 already exist
images/eu.51cf5495-1750-

images/eu.36be2e62-cd60-4850-b3f6-ff0cb61d11fc already exist
images/eu.72a18e67-1498-4a83-ac90-2ce0ee0d6e96 already exist
images/eu.b623afa0-8d84-4af3-8035-d0af865a96fd already exist
images/eu.c1f36e86-6169-49b5-a768-3106877595ca already exist
images/eu.6b62f237-7795-43d6-ac12-827fc33abc93 already exist
images/eu.59480baf-463f-4f38-93f4-604f8cb3f2d7 already exist
images/eu.f86a5449-e268-4eb5-b735-0c4b618ae840 already exist
images/eu.99be6e51-be21-4539-8090-5fdd9a34b17a already exist
images/eu.5e61054f-c6c8-44a2-ab76-2a7979fbb685 already exist
images/eu.ed25c201-ae29-47fc-8921-1fca880ae9b0 already exist
images/eu.5bc3b033-3c2d-4a22-8d78-1243949dcea5 already exist
images/eu.25bf781a-f4ac-46bc-aa30-b7cf53f09408 already exist
images/eu.171a0c85-c77d-421a-b93d-ba0965cd0cf5 already exist
images/eu.8898d082-750a-4525-9bbe-b4d8ec1297ac already exist
images/eu.3dbfc566-f99c-4c0e-a0a8-bc01c6fde726 already exist
images/eu.0124d977-d32d-4f97-af67-84fb69ab4909 already exist
images/eu.ff0a90a3-16a0-

images/eu.ec4c0430-6575-4f32-8599-da825102b8e6 already exist
images/eu.1e53df99-3886-447c-9bd4-b5d2f736f32b already exist
images/eu.8a5a2e0c-557f-464a-b37b-652434fb7ea1 already exist
images/eu.323ab0ad-f46d-4eb8-868c-f330a7a01fd0 already existimages/eu.e017ae31-ec46-403c-8c74-b45e783304d9 already exist

images/eu.e6922050-eac3-41cf-bf98-3e46cfebf50b already exist
images/eu.3cd9ce78-1ad4-40df-a1b0-c92438ea9d3e already exist
images/eu.57501f40-678c-4cec-961e-23549293254a already exist
images/eu.9d391ffc-ccf1-4767-8da6-77bc56f3d916 already exist
images/eu.7d791ba6-a37c-43b4-b8d8-25aea7147a5a already exist
images/eu.e42edffb-abe4-4724-a9cc-eb74d5f1f62d already exist
images/eu.24037579-d5da-4ba9-ba5c-2ef7bddfc4ae already exist
images/eu.5bd5f5c1-c0d4-49b8-9e11-4ac33b5efa99 already exist
images/eu.dec49c33-830f-4fe3-b8c7-2fc90bad80bc already exist
images/eu.72ebb48a-5421-460d-8a56-f03d0e565710 already exist
images/eu.ffc4e982-748d-47f6-9d4e-a11c1eb41ba9 already exist
images/eu.47303e3d-383d-

images/eu.3947c876-1bb2-4270-ab73-1f06b95f8517 already exist
images/eu.28a89c43-417e-40f2-b887-aa06d7b63f9c already exist
images/eu.4ddfb317-c5bd-4124-8671-116dd5d951b4 already exist
images/eu.b12cd21b-938a-4f58-b551-cc07cbdbd1bf already exist
images/eu.d99e9737-f0ea-4411-b243-55ccead50490 already exist
images/eu.820b8049-18e1-4703-8f45-e4f4f5cb56f5 already exist
images/eu.ba8a99c6-f3ed-41d1-a645-c68018a7b4f9 already exist
images/eu.ca4d8fc1-6697-4173-b2a9-cf3b6ba8ae31 already exist
images/eu.dcf761eb-3ee5-4433-b49a-5b25ca97e45a already exist
images/eu.1f5ade99-e50d-4e7e-affe-0effd1c11cbd already exist
images/eu.44e7d08e-9661-4d07-912a-1254c201c75f already exist
images/eu.a3e9ccf2-90c6-4838-9c42-19c6f546ab5b already exist
images/eu.49ff2f79-4087-4147-a5a9-7966c66bc537 already exist
images/eu.4196b491-5655-4c82-b689-656eb9fef31a already exist
images/eu.0fa9017b-64e3-4f48-b1ae-4a5d68c5d917 already exist
images/eu.c234a8db-2382-4b32-ab3b-68e71f79d01b already exist
images/eu.bf09bc50-b8a1-

In [27]:
len(image_hashes)

2000

In [28]:
image_hashes = dict(image_hashes)

In [29]:
processed = set()
to_check = []

In [30]:
for suite in tqdm(all_suits):
    suite_id = suite['id']
    if suite_id in processed:
        print(f'aready processed {suite_id}')
        continue

    task = suite['tasks'][0]
    solution = suite['solutions'][0]

    output_values = solution['output_values']

    output_class = output_values['class']

    if output_class == 'other':
        util.reject(suite_id, 'wrong item type')
        processed.add(suite_id)
        continue

    image = output_values['image']
    result = image_hashes[image] #util.download_image_and_calc_hashes(image)
    
    if result is None:
        util.reject(suite_id, 'problems with image')
        processed.add(suite_id)
        continue
        
    hashes, image_url = result

    if has_duplicates(hashes):
        util.reject(suite_id, 'duplicate')
        processed.add(suite_id)
        continue

    save_hashes(image, hashes)
    
    to_check.append({
        'image_url': image_url,
        'worker_id': suite['user_id'],
        'assignment_id': suite_id,
        'attachment_id': image
    })
    
    processed.add(suite_id)

  0%|          | 0/2000 [00:00<?, ?it/s]

00022bd050--6382ff7b0f6d6901eaf33bc4 REJECTED wrong item type
00022bd050--6382ff925d202a7b332349f0 REJECTED wrong item type
00022bd050--6382ffaa36abb2368ab2dc02 REJECTED wrong item type
00022bd050--6382ffb70f6d6901eaf33d06 REJECTED wrong item type
00022bd050--6383001dcfe9b7105c121924 REJECTED wrong item type
00022bd050--63830034d7580c45273ee7f5 REJECTED wrong item type
00022bd050--63830053e5eff94192e7393a REJECTED wrong item type
00022bd050--63830071d018bb1bf4aed8e3 REJECTED wrong item type
00022bd050--638300801518f405ad70f4ec REJECTED wrong item type
00022bd050--638300891518f405ad70f548 REJECTED wrong item type
00022bd050--638300b0ba00f040b9d40651 REJECTED wrong item type
00022bd050--638300d42d02d31945d869ea REJECTED wrong item type
00022bd050--638300eac6378a7992a4c7b1 REJECTED wrong item type
00022bd050--6383013236abb2368ab2e8a2 REJECTED wrong item type
00022bd050--6383014d0b8bed5c69be9507 REJECTED wrong item type
00022bd050--6383016365de190a41a5a778 REJECTED wrong item type
00022bd0

00022bd050--6383074ccfe9b7105c1265df REJECTED wrong item type
00022bd050--6383074ee5eff94192e7a750 REJECTED wrong item type
00022bd050--6383074ff9479e6276aa236a REJECTED wrong item type
00022bd050--6383075e8bf49c70832ac461 REJECTED wrong item type
00022bd050--638307631518f405ad711bbd REJECTED wrong item type
00022bd050--63830772d29cf2317031532e REJECTED wrong item type
00022bd050--638307772d02d31945d89880 REJECTED wrong item type
00022bd050--6383077d997f383b5a761e60 REJECTED wrong item type
00022bd050--638307844b57274dcf57fe19 REJECTED wrong item type
00022bd050--6383079dcfe9b7105c12682a REJECTED wrong item type
00022bd050--638307b48416e463827dbbd7 REJECTED wrong item type
00022bd050--638307c765de190a41a5d731 REJECTED wrong item type
00022bd050--638307ceb5a4e360698c5706 REJECTED wrong item type
00022bd050--638307d48bf49c70832ac7a8 REJECTED wrong item type
00022bd050--638307dec6378a7992a4fb82 REJECTED wrong item type
00022bd050--638307e0726a790e7ede40e8 REJECTED wrong item type
00022bd0

00022bd050--63830b9493facc7f211654f1 REJECTED wrong item type
00022bd050--63830b9965de190a41a61c52 REJECTED wrong item type
00022bd050--63830b9ac84cb8173bcccaaf REJECTED wrong item type
00022bd050--63830ba39d2ef67100f54c1c REJECTED wrong item type
00022bd050--63830bad0b8bed5c69bf00f8 REJECTED wrong item type
00022bd050--63830bb15d202a7b3323acac REJECTED wrong item type
00022bd050--63830bb965de190a41a61d6e REJECTED wrong item type
00022bd050--63830bbdaa1cad2a3f62c52b REJECTED wrong item type
00022bd050--63830be165de190a41a61eff REJECTED wrong item type
00022bd050--63830bf79d2ef67100f54e35 REJECTED wrong item type
00022bd050--63830c150b8bed5c69bf0332 REJECTED wrong item type
00022bd050--63830c152d02d31945d8ba5a REJECTED wrong item type
00022bd050--63830c22770f0c36ad6c8ae7 REJECTED wrong item type
00022bd050--63830c3661f48611858cd025 REJECTED wrong item type
00022bd050--63830c36aa1cad2a3f62c87c REJECTED wrong item type
00022bd050--63830c4a20f50d008d533570 REJECTED wrong item type
00022bd0

## Now let's upload this to Toloka!

In [32]:
len(to_check)

1664

In [35]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [36]:
pool_verify_id = 36426484
pool_upload_url = 'api/v1/task-suites?async_mode=false&open_pool=true'

In [37]:
batches = make_batches(to_check, 10)

In [38]:
len(batches)

167

In [39]:
for batch in batches:
    tasks = [{'input_values': d} for d in batch]

    data = {
        'pool_id': pool_verify_id,
        'tasks': tasks,
        "overlap": 5,
    }

    resp = util.toloka_post(pool_upload_url, data)
    print(resp)
    print(resp.json())

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '388795d6-9a76-41a3-8612-1359467bf526', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.5b35e616-208d-495f-99ad-6454955feb32.jpg', 'worker_id': 'f81c7bfde505fd77822689cdc3186315', 'assignment_id': '00022bd050--6382ff700b8bed5c69be888f', 'attachment_id': 'eu.5b35e616-208d-495f-99ad-6454955feb32'}}, {'id': 'fb3bf234-7351-4534-ad21-f023cdfad3c6', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c703be96-7957-4844-8eac-ad6b726a3ceb.jpg', 'worker_id': '2a91bc6e684c38de8041bfab2bf9a034', 'assignment_id': '00022bd050--6382ff7bc84cb8173bcc7e7d', 'attachment_id': 'eu.c703be96-7957-4844-8eac-ad6b726a3ceb'}}, {'id': '871714e9-abb0-42c2-aa53-abfd5248392f', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.6a1a87ba-5690-40ac-83fe-62f9992ef5ce.jpg', 'worker_id': 'dc081

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '38ed89f3-1a77-4916-b66c-0f20f73c6a34', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.0c6f3cb7-a9d3-416a-a991-5c7c2cbf5760.jpg', 'worker_id': 'ef5a57daa95e440c5ddacda75d143bb6', 'assignment_id': '00022bd050--638300082d02d31945d86453', 'attachment_id': 'eu.0c6f3cb7-a9d3-416a-a991-5c7c2cbf5760'}}, {'id': '0edd18bd-9b02-458b-a278-ab4f4e9e36a7', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.19862662-5143-4ea4-abfe-5bffe1a194b5.jpg', 'worker_id': '4f2a074151751323f767be44914d1f27', 'assignment_id': '00022bd050--6383000dabf3e72b4334657c', 'attachment_id': 'eu.19862662-5143-4ea4-abfe-5bffe1a194b5'}}, {'id': '16bdaaa1-177e-4f57-a4b5-1d05573e2d30', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c1280de1-35ab-4ee1-844a-67bdbe9f2c2b.jpg', 'worker_id': '72921

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '79c3c9c7-b82d-4340-99a2-700cd6aaea42', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.43359693-4d96-412a-ba2c-588b570df125.jpg', 'worker_id': 'b5dcf5e5d2a81956ceef94299ff25bfe', 'assignment_id': '00022bd050--638300a55674562d26041953', 'attachment_id': 'eu.43359693-4d96-412a-ba2c-588b570df125'}}, {'id': '65c45d7a-fef6-4c8d-a29c-2a4d60be3bbe', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.73a0a67f-52b8-4087-b14a-caa267177a6c.jpg', 'worker_id': 'f8ef0f7fc856fbee28e7f04993efb89a', 'assignment_id': '00022bd050--638300a5e7077310a7d57f68', 'attachment_id': 'eu.73a0a67f-52b8-4087-b14a-caa267177a6c'}}, {'id': '539779cb-78b7-4965-85df-174aa38774e0', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.1ab0d028-e57a-4795-be84-6a2c56135c4f.jpg', 'worker_id': 'b8237

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '273b33ae-f2eb-48d7-86ee-95aec3690db0', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.7914e693-049d-41c5-a14f-555f1dc02005.jpg', 'worker_id': 'ff8ea4b260cb388fa4071ca445f61aa6', 'assignment_id': '00022bd050--63830105e83ce12cf00bc7bb', 'attachment_id': 'eu.7914e693-049d-41c5-a14f-555f1dc02005'}}, {'id': '4aff8eb3-6e76-4512-8143-2175243b247c', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.67ff71aa-fc58-4aef-ad98-a8d2347eca1c.jpg', 'worker_id': '1b71ea1634e441ec2e355c4449707d17', 'assignment_id': '00022bd050--638301080bdf720fa975a695', 'attachment_id': 'eu.67ff71aa-fc58-4aef-ad98-a8d2347eca1c'}}, {'id': '416df28a-d78a-4cce-b966-02520104f91d', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.1705a136-c149-4500-af1d-e4e28eae87a7.jpg', 'worker_id': '0ae06

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '385a5578-e3cc-4379-884e-3423692d288f', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.93477a19-edd1-43a9-bff5-7d8b61d5ea25.jpg', 'worker_id': '3a069324a06c1e2c0f9827564e91b3df', 'assignment_id': '00022bd050--638301650f6d6901eaf34552', 'attachment_id': 'eu.93477a19-edd1-43a9-bff5-7d8b61d5ea25'}}, {'id': '63a9dfc6-37c2-4619-b6e0-26e9b0a8dd94', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.34afacf0-a9c4-4564-9bd0-ce5f0e73d9d1.jpg', 'worker_id': '1f8cdb9819d6d003147e8e4565ec257e', 'assignment_id': '00022bd050--638301668c39c02e11a3a508', 'attachment_id': 'eu.34afacf0-a9c4-4564-9bd0-ce5f0e73d9d1'}}, {'id': '93d26d84-8275-4456-b714-1e4513e432d6', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.cf547526-f09d-48eb-97c1-9022d7fb32f5.jpg', 'worker_id': '0ae06

...

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '2c3e043e-0d33-42e3-b534-52463c642b2e', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.cc35d15f-5389-432c-ac7b-7294902372d5.jpg', 'worker_id': 'a46bbb57c7ba6bfa6802d02006f4dba5', 'assignment_id': '00022bd050--63830dbbe83ce12cf00c19ab', 'attachment_id': 'eu.cc35d15f-5389-432c-ac7b-7294902372d5'}}, {'id': 'abad36a9-1583-419d-b0d2-38cb1901f8ac', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.724dcfdb-92ae-4892-9ec4-c1395a220817.jpg', 'worker_id': 'f9124e4ac24e86bf406f1b6b08f280ac', 'assignment_id': '00022bd050--63830dbecfe9b7105c12cbb8', 'attachment_id': 'eu.724dcfdb-92ae-4892-9ec4-c1395a220817'}}, {'id': '0ad5af23-02ed-4a8c-9f69-22b9151ef02a', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.025e7b03-ac1b-427d-b25e-7a1534bbba9b.jpg', 'worker_id': '7b476

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '63da28f2-eff4-4201-b8c1-2ecbc818fd31', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.aa93b470-1505-4fb2-98df-506b4f4fe5dc.jpg', 'worker_id': 'fa866e18914b72c557e401c7b5cfbe45', 'assignment_id': '00022bd050--63830de8997f383b5a764cc0', 'attachment_id': 'eu.aa93b470-1505-4fb2-98df-506b4f4fe5dc'}}, {'id': '20b4aa8a-59b0-4ea0-8b6e-645c2ec50f99', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.40a3a235-3dc9-4b8f-9a19-851b9b5aa5f6.jpg', 'worker_id': '07d80baccc51be7944a8940d5f8a1adc', 'assignment_id': '00022bd050--63830dea60ea55120a275828', 'attachment_id': 'eu.40a3a235-3dc9-4b8f-9a19-851b9b5aa5f6'}}, {'id': 'de381ee7-bacf-4c1b-b5ba-ce100d3d6ddb', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.f3ad379a-444e-44dc-ab33-a757e1d5a88b.jpg', 'worker_id': 'cdb05

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '3f04a3f6-8f6f-4405-a739-a9f38abb737a', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.4196b491-5655-4c82-b689-656eb9fef31a.jpg', 'worker_id': '7462da342d0ff13d0f729f4c2d7d27c6', 'assignment_id': '00022bd050--63830e2bc6378a7992a5357b', 'attachment_id': 'eu.4196b491-5655-4c82-b689-656eb9fef31a'}}, {'id': 'e9fb8449-2522-488a-984c-baaf60bea25f', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.0fa9017b-64e3-4f48-b1ae-4a5d68c5d917.jpg', 'worker_id': 'aebc6df04f87b2dc5922360301c9ac13', 'assignment_id': '00022bd050--63830e30d4735467adbeb63c', 'attachment_id': 'eu.0fa9017b-64e3-4f48-b1ae-4a5d68c5d917'}}, {'id': '1e0edf52-9499-4161-909e-0154b2df4b35', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.c234a8db-2382-4b32-ab3b-68e71f79d01b.jpg', 'worker_id': 'e332e

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '248d146f-a81a-495e-9833-ca0d7694a4e2', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.584f8dd9-2278-4295-b33b-62e680ee81e6.jpg', 'worker_id': 'ef09ad43d025f17401739dc71b648a6a', 'assignment_id': '00022bd050--63830e65c7029237f68d41d7', 'attachment_id': 'eu.584f8dd9-2278-4295-b33b-62e680ee81e6'}}, {'id': '41b87061-3d54-4512-b85c-95070c701da2', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.96ed5ebf-3690-444f-8d0a-9812f4552449.jpg', 'worker_id': '428fa436974ce0138e53182b57ea368d', 'assignment_id': '00022bd050--63830e6861f48611858cddc5', 'attachment_id': 'eu.96ed5ebf-3690-444f-8d0a-9812f4552449'}}, {'id': 'c9004ed0-64bd-4dab-9c53-36e645411021', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.9d20bfa6-6801-4727-a234-533716fee51e.jpg', 'worker_id': 'e332e

<Response [201]>
{'pool_id': '36426484', 'tasks': [{'id': '47db8845-3b55-4b74-a3e6-d7d00e145a3c', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.30e5d87a-1103-40f4-bea0-67125bb4426e.jpg', 'worker_id': '3b3bdf4cf0813fb9c6972ec04598db98', 'assignment_id': '00022bd050--6383108ce5eff94192e7fa8e', 'attachment_id': 'eu.30e5d87a-1103-40f4-bea0-67125bb4426e'}}, {'id': 'cb074f6e-dd91-478b-84b2-b483e8f200b1', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.69844c6f-fdc1-4db0-84bc-7d8a5cfc997e.jpg', 'worker_id': '3b3bdf4cf0813fb9c6972ec04598db98', 'assignment_id': '00022bd050--638310ae8916f97efd07fe5c', 'attachment_id': 'eu.69844c6f-fdc1-4db0-84bc-7d8a5cfc997e'}}, {'id': '61482f54-73fa-4b74-83d7-ec8188e7e544', 'input_values': {'image_url': 'https://toloka-kitchenware.s3.eu-west-1.amazonaws.com/36425808/thumbnails/eu.90f14201-4238-415c-97c6-d2d5f2d9668f.jpg', 'worker_id': '3b3bd

## Accept/reject the original images

In [12]:
pool_verify_id = 36426484

In [13]:
verify_suits, last_id = get_available_suits(
    pool_verify_id,
    last_id='00022bd2f4--63796a1785bacb70b68343aa',
    status='ACCEPTED'
)

added 100, now have 100 suits (100 unique)
added 100, now have 200 suits (200 unique)
added 100, now have 300 suits (300 unique)
added 100, now have 400 suits (400 unique)
added 100, now have 500 suits (500 unique)
added 100, now have 600 suits (600 unique)
added 100, now have 700 suits (700 unique)
added 100, now have 800 suits (800 unique)
added 35, now have 835 suits (835 unique)


In [14]:
last_id

'00022bd2f4--6384a5e045eaf3496bc9c218'

In [15]:
data = []

for suite in verify_suits:
    for task, solution in zip(suite['tasks'], suite['solutions']):
        input_values = task['input_values']
        output_values = solution['output_values']
    
        row = input_values.copy()
        row.update(output_values)
        row['verifier'] = suite['user_id']
        data.append(row)

In [16]:
len(data)

8320

In [17]:
df = pd.DataFrame(data)

In [18]:
df.verdict.value_counts()

yes                   7264
no-not-kitchenware     535
no-multiple-items      287
no-other               185
no-internet             34
no-screenshot           15
Name: verdict, dtype: int64

In [19]:
gb = df.groupby('assignment_id')

In [20]:
all_results = []

for assignment_id, df_group in gb:
    classes = Counter(df_group['class'])
    verdicts = Counter(df_group['verdict'])
    
    most_common_verdict, _ = verdicts.most_common(1)[0]
    most_common_class, num_votes = classes.most_common(1)[0]

    agreement = num_votes / len(df_group)

    record = {
        'assignment_id': assignment_id,
        'agreement': agreement,
        'image': df_group.iloc[0].image_url,
        'image_id': df_group.iloc[0].attachment_id,
        'most_common_class': most_common_class,
        'most_common_verdict': most_common_verdict,
        'verdicts': verdicts,
        'classes': classes,
    }
    
    all_results.append(record)

In [21]:
df_agreements = pd.DataFrame(all_results)

In [22]:
df_agreements

,assignment_id,agreement,image,image_id,most_common_class,most_common_verdict,verdicts,classes
0,00022bd050--6382ff700b8bed5c69be888f,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.5b35e616-208d-495f-99ad-6454955feb32,other,no-other,"{'no-other': 3, 'no-not-kitchenware': 2}",{'other': 5}
1,00022bd050--6382ff7bc84cb8173bcc7e7d,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.c703be96-7957-4844-8eac-ad6b726a3ceb,glass,yes,{'yes': 5},{'glass': 5}
2,00022bd050--6382ff86d1d08f0d875b9d25,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.6a1a87ba-5690-40ac-83fe-62f9992ef5ce,glass,yes,{'yes': 5},{'glass': 5}
3,00022bd050--6382ff8ee5eff94192e7336c,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.f91f6108-a822-4ada-9f1c-759b606ad02c,plate,yes,{'yes': 5},{'plate': 5}
4,00022bd050--6382ff90f9479e6276a9f114,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.4fa71998-aeda-4911-a5fa-359bf2f81268,knife,yes,{'yes': 5},{'knife': 5}
...,...,...,...,...,...,...,...,...
1659,00022bd050--6383124f0f6d6901eaf3b8ec,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.9f871bd9-4b76-414f-b8c4-add61ccc8178,plate,yes,{'yes': 5},{'plate': 5}
1660,00022bd050--63831250d6be7f67b7880513,0.8,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.d26e2844-d4f3-4ee6-81ba-87cf4aeceb68,glass,yes,{'yes': 5},"{'glass': 4, 'other': 1}"
1661,00022bd050--638312a10f6d6901eaf3baca,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.446b59a2-f48a-4d35-a291-089cc164833a,other,no-not-kitchenware,"{'no-not-kitchenware': 3, 'yes': 2}",{'other': 5}
1662,00022bd050--638313783cf7153391fbe785,1.0,https://toloka-kitchenware.s3.eu-west-1.amazon...,eu.c365d349-8ffd-45c2-af66-8a9d21ec04d0,other,no-not-kitchenware,"{'no-not-kitchenware': 4, 'no-other': 1}",{'other': 5}


In [25]:
good = (df_agreements.most_common_verdict == 'yes') & \
    (df_agreements.most_common_class != 'other') & \
    (df_agreements.agreement >= 0.5)

In [26]:
sum(good) / len(good)

0.8695913461538461

In [27]:
df_agreements['final_verdict'] = 'ACCEPT'
df_agreements.loc[~good, 'final_verdict'] = 'REJECT'

In [28]:
df_agreements['comment'] = 'good job'
df_agreements.loc[~good, 'comment'] = "doesn't satisfy the requrements"

In [29]:
def process_agreements(row):
    assignment_id = row['assignment_id']
    verdict = row['final_verdict']
    comment = row['comment']

    if verdict == 'ACCEPT':
        util.accept(assignment_id)
    else:
        util.reject(assignment_id, comment)

In [30]:
rows = df_agreements.to_dict(orient='records')

In [31]:
_ = util.map_progress(pool, rows, process_agreements)

  0%|          | 0/1664 [00:00<?, ?it/s]

00022bd050--6382ffa0e7077310a7d5643d ACCEPTED good job
00022bd050--6382ffa24c335537e521d339 ACCEPTED good job
00022bd050--6382ff86d1d08f0d875b9d25 ACCEPTED good job
00022bd050--6382ff9865de190a41a59b5a ACCEPTED good job
00022bd050--6382ff8ee5eff94192e7336c ACCEPTED good job
00022bd050--6382ffb78c39c02e11a395db ACCEPTED good job
00022bd050--6382ff90f9479e6276a9f114 ACCEPTED good job
00022bd050--6382ff7bc84cb8173bcc7e7d ACCEPTED good job
00022bd050--6382ffba997f383b5a75eb1b ACCEPTED good job
00022bd050--6382ff700b8bed5c69be888f REJECTED doesn't satisfy the requrements
00022bd050--6382ff9560ea55120a26dd4c ACCEPTED good job
00022bd050--6382ffbfd1d08f0d875b9e7b ACCEPTED good job
00022bd050--6382ffc7e5eff94192e735a0 ACCEPTED good job
00022bd050--6382ffcfba00f040b9d40179 REJECTED doesn't satisfy the requrements
00022bd050--6382ffdbd55ea30be688f952 REJECTED doesn't satisfy the requrements
00022bd050--6382ffcae7077310a7d568ba ACCEPTED good job
00022bd050--6382fff28916f97efd078198 ACCEPTED good 

00022bd050--638301b28bf49c70832a96e1 ACCEPTED good job
00022bd050--638301b955130e402a667364 ACCEPTED good job
00022bd050--638301b7d4735467adbe4719 ACCEPTED good job
00022bd050--638301ae65de190a41a5a92e ACCEPTED good job
00022bd050--638301b9c09e7b0c14e27b8b ACCEPTED good job
00022bd050--638301bb8916f97efd078cd9 REJECTED doesn't satisfy the requrements
00022bd050--638301cbf3f5c54b222b1b95 ACCEPTED good job
00022bd050--638301c5ba00f040b9d40f7d ACCEPTED good job
00022bd050--638301c55674562d26042046 ACCEPTED good job
00022bd050--638301bfd4735467adbe4726 REJECTED doesn't satisfy the requrements
00022bd050--638301bed7580c45273eef21 ACCEPTED good job
00022bd050--638301c03cf7153391fb2fc0 REJECTED doesn't satisfy the requrements
00022bd050--638301cbe7077310a7d59923 ACCEPTED good job
00022bd050--638301c960ea55120a26ee4b ACCEPTED good job
00022bd050--638301cbc6378a7992a4cd34 ACCEPTED good job
00022bd050--638301d27cea4c44aecb29ff ACCEPTED good job
00022bd050--638301d10bdf720fa975abed ACCEPTED good 

00022bd050--638302f8f3f5c54b222b26b0 ACCEPTED good job
00022bd050--638302ff196e87056e82efb0 ACCEPTED good job
00022bd050--63830308c09e7b0c14e282d7 ACCEPTED good job
00022bd050--63830310e83ce12cf00bd496 ACCEPTED good job
00022bd050--638303083cf7153391fb5a9c REJECTED doesn't satisfy the requrements
00022bd050--638302ffba00f040b9d416f9 ACCEPTED good job
00022bd050--638303118c39c02e11a3b141 ACCEPTED good job
00022bd050--63830312abf3e72b43348637 ACCEPTED good job
00022bd050--6383030c4b57274dcf57dfec ACCEPTED good job
00022bd050--63830311e5eff94192e75552 ACCEPTED good job
00022bd050--638303018416e463827d946c ACCEPTED good job
00022bd050--638302fac7029237f68cdeec ACCEPTED good job
00022bd050--638303230bdf720fa975b890 ACCEPTED good job
00022bd050--6383031be83ce12cf00bd50d ACCEPTED good job
00022bd050--638303188c39c02e11a3b14e ACCEPTED good job
00022bd050--63830314f9479e6276aa08de ACCEPTED good job
00022bd050--6383031c8c39c02e11a3b1b7 ACCEPTED good job
00022bd050--6383031faa1cad2a3f62855b ACCEP

00022bd050--638304462d02d31945d88264 ACCEPTED good job
00022bd050--6383044eba00f040b9d41f6b ACCEPTED good job
00022bd050--63830453c09e7b0c14e28ba0 REJECTED doesn't satisfy the requrements
00022bd050--6383045db5a4e360698c3ec0 ACCEPTED good job
00022bd050--6383045dd018bb1bf4aef2bd REJECTED doesn't satisfy the requrements
00022bd050--6383045e7a6899134e1ad914 ACCEPTED good job
00022bd050--6383045b726a790e7eddff5b ACCEPTED good job
00022bd050--63830456e5eff94192e76006 REJECTED doesn't satisfy the requrements
00022bd050--63830461997f383b5a760849 ACCEPTED good job
00022bd050--6383045be5eff94192e76023 ACCEPTED good job
00022bd050--6383045961f48611858c9e57 ACCEPTED good job
00022bd050--6383046655130e402a668552 ACCEPTED good job
00022bd050--638304694c335537e521fef9 ACCEPTED good job
00022bd050--6383046a55130e402a66856b REJECTED doesn't satisfy the requrements
00022bd050--6383046bc6378a7992a4e146 ACCEPTED good job
00022bd050--6383046d8916f97efd07a3c5 ACCEPTED good job
00022bd050--6383046e997f383b

...

00022bd050--638309721518f405ad712894 ACCEPTED good job
00022bd050--63830970abf3e72b4334b768 ACCEPTED good job
00022bd050--63830967a0908d7db4bf5dd9 ACCEPTED good job
00022bd050--63830970c7029237f68d17c2 ACCEPTED good job
00022bd050--63830978e83ce12cf00bf9fb ACCEPTED good job
00022bd050--6383096a7a6899134e1b0273 ACCEPTED good job
00022bd050--638309670b8bed5c69bef183 ACCEPTED good job
00022bd050--6383097bb5a4e360698c62ba ACCEPTED good job
00022bd050--63830979c6378a7992a5064e REJECTED doesn't satisfy the requrements
00022bd050--6383097de7077310a7d5d0f6 ACCEPTED good job
00022bd050--63830901cfe9b7105c1276c7 REJECTED doesn't satisfy the requrements
00022bd050--6383097e65de190a41a5f07e ACCEPTED good job
00022bd050--638309895674562d260485de ACCEPTED good job
00022bd050--63830988aa1cad2a3f62b477 ACCEPTED good job
00022bd050--6383097f0b8bed5c69bef235 ACCEPTED good job
00022bd050--6383097f1518f405ad712909 ACCEPTED good job
00022bd050--6383098165de190a41a5f0ab ACCEPTED good job
00022bd050--6383098

00022bd050--63830b03d018bb1bf4af1d4a REJECTED doesn't satisfy the requrements
00022bd050--63830b0cf9479e6276aa3f46 ACCEPTED good job
00022bd050--63830aff1518f405ad7132ea ACCEPTED good job
00022bd050--63830b16b5a4e360698c6d95 ACCEPTED good job
00022bd050--63830b050b8bed5c69befc5f ACCEPTED good job
00022bd050--63830b158bf49c70832ae800 ACCEPTED good job
00022bd050--63830b1b3cf7153391fb9e05 REJECTED doesn't satisfy the requrements
00022bd050--63830b1ab5a4e360698c6da2 ACCEPTED good job
00022bd050--63830b1c0b8bed5c69befce9 ACCEPTED good job
00022bd050--63830b1fe83ce12cf00c05a0 ACCEPTED good job
00022bd050--63830b2fabf3e72b4334c3d9 ACCEPTED good job
00022bd050--63830b29a0908d7db4bf6a21 ACCEPTED good job
00022bd050--63830b31d1d08f0d875bf9fe REJECTED doesn't satisfy the requrements
00022bd050--63830b2f8416e463827dd86b ACCEPTED good job
00022bd050--63830b34b5a4e360698c6eae ACCEPTED good job
00022bd050--63830b29e83ce12cf00c05f4 ACCEPTED good job
00022bd050--63830b330b8bed5c69befd9a REJECTED doesn

00022bd050--63830c42d1d08f0d875c0295 ACCEPTED good job
00022bd050--63830c464c335537e522468d ACCEPTED good job
00022bd050--63830c488bf49c70832af0cd ACCEPTED good job
00022bd050--63830c4f3cf7153391fba8e6 ACCEPTED good job
00022bd050--63830c544c335537e5224701 ACCEPTED good job
00022bd050--63830c512d02d31945d8bc55 ACCEPTED good job
00022bd050--63830c51d4735467adbe9df1 ACCEPTED good job
00022bd050--63830c3e65de190a41a62180 ACCEPTED good job
00022bd050--63830c54a0908d7db4bf733b ACCEPTED good job
00022bd050--63830c5d4b57274dcf581c96 REJECTED doesn't satisfy the requrements
00022bd050--63830c59d29cf23170318538 ACCEPTED good job
00022bd050--63830c5dcfe9b7105c12b17c ACCEPTED good job
00022bd050--63830c5961f48611858cd0fd REJECTED doesn't satisfy the requrements
00022bd050--63830c607cea4c44aecb726a REJECTED doesn't satisfy the requrements
00022bd050--63830c6155130e402a66bf41 ACCEPTED good job
00022bd050--63830c5e0b8bed5c69bf0579 ACCEPTED good job
00022bd050--63830c6155130e402a66bf47 ACCEPTED good 

00022bd050--63830d3b770f0c36ad6c968c ACCEPTED good job
00022bd050--63830d40aabcbc3b0444a759 REJECTED doesn't satisfy the requrements
00022bd050--63830d4736abb2368ab359b8 ACCEPTED good job
00022bd050--63830d4236abb2368ab3599b ACCEPTED good job
00022bd050--63830d40ba00f040b9d45ba1 ACCEPTED good job
00022bd050--63830d430b8bed5c69bf0c69 ACCEPTED good job
00022bd050--63830d42c84cb8173bccd643 ACCEPTED good job
00022bd050--63830d41d29cf23170318bb3 ACCEPTED good job
00022bd050--63830d442d02d31945d8c562 ACCEPTED good job
00022bd050--63830d46d018bb1bf4af2ad1 ACCEPTED good job
00022bd050--63830d49196e87056e834a8a ACCEPTED good job
00022bd050--63830d498416e463827dea52 ACCEPTED good job
00022bd050--63830d4b36abb2368ab359d7 ACCEPTED good job
00022bd050--63830d51726a790e7ede72da ACCEPTED good job
00022bd050--63830d4c20f50d008d533dca REJECTED doesn't satisfy the requrements
00022bd050--63830d4d5d202a7b3323bcea ACCEPTED good job
00022bd050--63830d5355130e402a66c63c ACCEPTED good job
00022bd050--63830d5

00022bd050--63830e2020f50d008d53442c ACCEPTED good job
00022bd050--63830e1f5674562d2604ae67 ACCEPTED good job
00022bd050--63830e2420f50d008d534462 ACCEPTED good job
00022bd050--63830e30d4735467adbeb63c ACCEPTED good job
00022bd050--63830e25e3e95c18828b42f7 ACCEPTED good job
00022bd050--63830e30d55ea30be68957a4 ACCEPTED good job
00022bd050--63830e3a4c335537e5225bb0 ACCEPTED good job
00022bd050--63830e32997f383b5a764efb ACCEPTED good job
00022bd050--63830e388416e463827df218 ACCEPTED good job
00022bd050--63830e389d2ef67100f56429 ACCEPTED good job
00022bd050--63830e3e60ea55120a275aa7 ACCEPTED good job
00022bd050--63830dd43cf7153391fbb428 ACCEPTED good job
00022bd050--63830e3a5674562d2604af55 ACCEPTED good job
00022bd050--63830e42c84cb8173bccddce ACCEPTED good job
00022bd050--63830e3b8416e463827df235 ACCEPTED good job
00022bd050--63830e44d7580c45273f3ca8 ACCEPTED good job
00022bd050--63830e4dc6378a7992a53697 ACCEPTED good job
00022bd050--63830e4b0b8bed5c69bf12ce ACCEPTED good job
00022bd050

In [32]:
df_agreements.to_csv('batch5.csv', index=False)